In [1]:
# in this script, we will produce spatial average feedbacks
# this saves computation time in future plotting scripts

#by: Ty Janoski
# updated: 06.04.20

In [2]:
# import statements
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def spatial_mean(ds,lat_bound=-90):
    """Take the mean over latitude and longitude dimensions,
    while weighting for latitude.

    PARAMETERS
    ----------
    datasets : list of xarray.Dataset
        A list of xarray datasets to return .

    RETURNS
    -------
    filelist : A list of xarray datasets retrieved from server.
    """
    area_of_int = ds.mean(dim='lon').sel(lat = slice(lat_bound, None))
    weights = (np.cos(np.deg2rad(area_of_int['lat']))/
               np.sum(np.cos(np.deg2rad(area_of_int['lat']))))
    return((area_of_int * weights).sum(dim='lat'))

In [4]:
models = ['ACCESS1-0','ACCESS1-3','CNRM-CM5','IPSL-CM5B-LR', 'GFDL-ESM2G',
         'MIROC-ESM', 'FGOALS-g2','bcc-csm1-1','BNU-ESM','CanESM2','CCSM4',
          'CSIRO-Mk3-6-0','FGOALS-s2','GFDL-CM3','inmcm4',
         'IPSL-CM5A-LR','MIROC5','MPI-ESM-LR','MPI-ESM-P','MRI-CGCM3','NorESM1-M']
for m in models:
    print(m)
    ts = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+m+'_dTs.nc',use_cftime=True).load()
    wv = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+m+'_wv.nc',use_cftime=True).load()
    planck = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+m+'_planck.nc',use_cftime=True).load()
    lapse = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+m+'_lapse.nc',use_cftime=True).load()
    alb = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+m+'_shell_albedo.nc',use_cftime=True).load()
    cloud = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+m+'_cloud.nc',use_cftime=True).load()
    atmos = xr.open_dataarray('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+m+'_atmos_conv.nc',use_cftime=True).load().to_dataset(name='atmos_conv')
    
    arc = xr.merge([spatial_mean(i,lat_bound=70) for i in [ts,wv,planck,lapse,alb,cloud,atmos]])
    glb = xr.merge([spatial_mean(i) for i in [ts,wv,planck,lapse,alb,cloud,atmos]])
        
    if m in ['BNU-ESM','CCSM4','CSIRO-Mk3-6-0','GFDL-CM3','MIROC5',
               'MPI-ESM-LR','MPI-ESM-P','MRI-CGCM3','NorESM1-M','ACCESS1-0','ACCESS1-3',
              'CNRM-CM5','FGOALS-g2','MIROC-ESM']:
        ocean = xr.open_dataarray('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+m+'_ocean_conv.nc',use_cftime=True).load()
        if(m in ['CCSM4','FGOALS-g2']):
            ocean['time'] = ts.time
        areacello = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/areacello_fx_'+m+'_*.nc',
                                     combine='by_coords').areacello
        weights_arc = areacello.where(areacello.lat>70) / areacello.where(areacello.lat>70).sum().sum()
        weights_glb = areacello / areacello.sum().sum()
        ocean_arc = (weights_arc * ocean).sum(axis=[0,1]).to_dataset(name='ml_conv')
        ocean_glb = (weights_glb * ocean).sum(axis=[0,1]).to_dataset(name='ml_conv')
        arc = xr.merge([arc,ocean_arc])
        glb = xr.merge([glb,ocean_glb])
    
    print(arc)
    print(glb)
    
    arc.to_netcdf('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+m+'_shell_arc_feedbacks.nc')
    glb.to_netcdf('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+m+'_shell_glb_feedbacks.nc')

ACCESS1-0
<xarray.Dataset>
Dimensions:      (time: 1800)
Coordinates:
  * time         (time) object 0300-01-16 12:00:00 ... 0449-12-16 12:00:00
Data variables:
    dTs          (time) float64 -2.148 1.832 -0.6739 ... 15.69 20.42 24.31
    wv_LWAS      (time) float64 -0.6372 -0.1257 -0.4374 ... 3.474 1.874 0.889
    wv_LWCS      (time) float64 -0.7117 -0.2108 -0.5215 ... 4.33 2.348 1.067
    wv_SWAS      (time) float64 -0.0002106 0.0004562 -0.0157 ... 0.01522 0.0
    wv_SWCS      (time) float64 -0.0002578 0.00262 -0.01264 ... 0.01877 0.0
    wv_NETAS     (time) float64 -0.6374 -0.1252 -0.4531 ... 3.894 1.889 0.889
    wv_NETCS     (time) float64 -0.712 -0.2082 -0.5341 ... 4.782 2.367 1.067
    planck_LWAS  (time) float64 5.176 -4.077 1.764 ... -40.36 -49.64 -57.18
    lapse_LWAS   (time) float64 -0.8058 0.6242 -2.019 ... 15.38 20.25 23.04
    albedo_SWAS  (time) float64 -0.000141 -0.01874 -0.2054 ... 1.716 0.03036 0.0
    albedo_SWCS  (time) float64 -0.0004029 -0.08106 -0.6891 ... 0.07

In [3]:
def spatial_mean(ds,lat_south=-91,lat_north=91):
    """Take the mean over latitude and longitude dimensions,
    while weighting for latitude.

    PARAMETERS
    ----------
    datasets : list of xarray.Dataset
        A list of xarray datasets to return .

    RETURNS
    -------
    filelist : A list of xarray datasets retrieved from server.
    """
    area_of_int = ds.mean(dim='lon').sel(lat = slice(lat_south, lat_north))
    weights = (np.cos(np.deg2rad(area_of_int['lat']))/
               np.sum(np.cos(np.deg2rad(area_of_int['lat']))))
    return((area_of_int * weights).sum(dim='lat'))

In [6]:
models = ['ACCESS1-0','ACCESS1-3','CNRM-CM5','IPSL-CM5B-LR', 'GFDL-ESM2G',
         'MIROC-ESM', 'FGOALS-g2','bcc-csm1-1','BNU-ESM','CanESM2','CCSM4',
          'CSIRO-Mk3-6-0','FGOALS-s2','GFDL-CM3','inmcm4',
         'IPSL-CM5A-LR','MIROC5','MPI-ESM-LR','MPI-ESM-P','MRI-CGCM3','NorESM1-M']
for m in models:
    print(m)
    ts = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+m+'_dTs.nc',use_cftime=True).load()
    wv = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+m+'_wv.nc',use_cftime=True).load()
    planck = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+m+'_planck.nc',use_cftime=True).load()
    lapse = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+m+'_lapse.nc',use_cftime=True).load()
    alb = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+m+'_albedo.nc',use_cftime=True).load()
    cloud = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+m+'_cloud.nc',use_cftime=True).load()
    atmos = xr.open_dataarray('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+m+'_atmos_conv.nc',use_cftime=True).load().to_dataset(name='atmos_conv')
    
    subarc = xr.merge([spatial_mean(i,lat_south=70) for i in [ts,wv,planck,lapse,alb,cloud,atmos]])
    trop = xr.merge([spatial_mean(i,lat_south=-30,lat_north=30) for i in [ts,wv,planck,lapse,alb,cloud,atmos]])
        
    if m in ['BNU-ESM','CCSM4','CSIRO-Mk3-6-0','GFDL-CM3','MIROC5',
               'MPI-ESM-LR','MPI-ESM-P','MRI-CGCM3','NorESM1-M','ACCESS1-0','ACCESS1-3',
              'CNRM-CM5','FGOALS-g2','MIROC-ESM']:
        ocean = xr.open_dataarray('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+m+'_ocean_conv.nc',use_cftime=True).load()
        if(m in ['CCSM4','FGOALS-g2']):
            ocean['time'] = ts.time
        areacello = xr.open_mfdataset('/dx07/tylerj/CMIP5_output/piControl/areacello_fx_'+m+'_*.nc',
                                     combine='by_coords').areacello
        weights_subarc = areacello.where(areacello.lat>60) / areacello.where(areacello.lat>60).sum().sum()
        weights_trop = areacello.where(abs(areacello.lat)<=30) / areacello.where(abs(areacello.lat)<=30).sum().sum()
        ocean_subarc = (weights_subarc * ocean).sum(axis=[0,1]).to_dataset(name='ml_conv')
        ocean_trop = (weights_trop * ocean).sum(axis=[0,1]).to_dataset(name='ml_conv')
        subarc = xr.merge([subarc,ocean_subarc])
        trop = xr.merge([trop,ocean_trop])
    
    print(subarc)
    print(trop)
    
    subarc.to_netcdf('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+m+'_subarc_feedbacks.nc')
    trop.to_netcdf('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+m+'_trop_feedbacks.nc')

ACCESS1-0
<xarray.Dataset>
Dimensions:      (time: 1800)
Coordinates:
  * time         (time) object 0300-01-16 12:00:00 ... 0449-12-16 12:00:00
Data variables:
    dTs          (time) float64 -2.148 1.832 -0.6739 ... 15.69 20.42 24.31
    wv_LWAS      (time) float64 -0.6372 -0.1257 -0.4374 ... 3.474 1.874 0.889
    wv_LWCS      (time) float64 -0.7117 -0.2108 -0.5215 ... 4.33 2.348 1.067
    wv_SWAS      (time) float64 -0.0002106 0.0004562 -0.0157 ... 0.01522 0.0
    wv_SWCS      (time) float64 -0.0002578 0.00262 -0.01264 ... 0.01877 0.0
    wv_NETAS     (time) float64 -0.6374 -0.1252 -0.4531 ... 3.894 1.889 0.889
    wv_NETCS     (time) float64 -0.712 -0.2082 -0.5341 ... 4.782 2.367 1.067
    planck_LWAS  (time) float64 5.176 -4.077 1.764 ... -40.36 -49.64 -57.18
    lapse_LWAS   (time) float64 -0.8058 0.6242 -2.019 ... 15.38 20.25 23.04
    albedo_SWAS  (time) float64 -0.0006216 -0.05722 -0.4644 ... 0.06601 0.0
    albedo_SWCS  (time) float64 -0.0007893 -0.09346 -0.7501 ... 0.08547 0

In [5]:
ocean_subarc

<xarray.Dataset>
Dimensions:  (time: 1800)
Coordinates:
  * time     (time) object 0001-01-16 12:00:00 ... 0150-12-16 12:00:00
Data variables:
    ml_conv  (time) float64 dask.array<chunksize=(1800,), meta=np.ndarray>